In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv1D,
    MaxPooling1D,
    LSTM,
    Bidirectional,
    Dropout,
    Flatten,
    Dense,
    Input,
)
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import metrics

from utils import load_dataset, train, auc, plot_roc
from einops import rearrange

<IPython.core.display.Javascript object>

In [2]:
NUM_OUTPUTS = 1

model = Sequential(
    [
        Input(shape=(1000, 4)),
        Conv1D(320, kernel_size=8, activation="relu"),
        MaxPooling1D(pool_size=4, strides=4),
        Dropout(0.2),
        Conv1D(480, kernel_size=8, activation="relu"),
        MaxPooling1D(pool_size=8, strides=8),
        Dropout(0.2),
        Flatten(data_format="channels_first"),
        Dense(512, activation="relu"),
        Dense(NUM_OUTPUTS, activation="softmax"),
    ],
    name="deepsea_full",
)

model.compile(optimizer=Adam(lr=1e-2), loss="binary_crossentropy", metrics=["acc"])
model.summary()

Model: "deepsea_full"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 993, 320)          10560     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 248, 320)          0         
_________________________________________________________________
dropout (Dropout)            (None, 248, 320)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 241, 480)          1229280   
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 30, 480)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 480)           0         
_________________________________________________________________
flatten (Flatten)            (None, 14400)            

<IPython.core.display.Javascript object>

In [3]:
es = EarlyStopping(monitor="val_loss", patience=10)

epochs = 250
validation_freq = 1
n = 2
file = f"MA0035_{n}000"

<IPython.core.display.Javascript object>

In [4]:
X_train, y_train, X_test, y_test = load_dataset(file, labels="binlabels")
# convert from shape (n, 4, 1000) to (n, 1000, 4)
X_train = rearrange(X_train, "w h c -> w c h")
X_test = rearrange(X_test, "w h c -> w c h")

model = train(
    dataset=(X_train, y_train, X_test, y_test),
    model=model,
    epochs=epochs,
    verbose=2,
    validation_freq=validation_freq,
    callbacks=[],
    batch_size=100,
)

y_pred = model.predict(X_test)
plot_roc(y_test, y_pred)
print(auc(y_test, y_pred))

Epoch 1/250
1000/1000 - 46s - loss: 1.0425 - acc: 0.5000 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 2/250
1000/1000 - 44s - loss: 0.6933 - acc: 0.5000 - val_loss: 0.6933 - val_acc: 0.5000
Epoch 3/250
1000/1000 - 45s - loss: 0.6932 - acc: 0.5000 - val_loss: 0.6944 - val_acc: 0.5000
Epoch 4/250
1000/1000 - 44s - loss: 0.6933 - acc: 0.5000 - val_loss: 0.6936 - val_acc: 0.5000
Epoch 5/250
1000/1000 - 44s - loss: 0.6932 - acc: 0.5000 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 6/250
1000/1000 - 44s - loss: 0.6933 - acc: 0.5000 - val_loss: 0.6934 - val_acc: 0.5000
Epoch 7/250
1000/1000 - 43s - loss: 0.6933 - acc: 0.5000 - val_loss: 0.6933 - val_acc: 0.5000
Epoch 8/250
1000/1000 - 46s - loss: 0.6933 - acc: 0.5000 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 9/250
1000/1000 - 46s - loss: 0.6932 - acc: 0.5000 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 10/250
1000/1000 - 47s - loss: 0.6933 - acc: 0.5000 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 11/250
1000/1000 - 52s - loss: 0.6933 - acc: 0.5000 -

KeyboardInterrupt: 

<IPython.core.display.Javascript object>

In [ ]:
model.save(f"../model_saves/evals/{model.name}-{n}000.h5")